In [124]:
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.io import fits
from scipy.optimize import linear_sum_assignment

In [125]:
# not useful rn but saved in case i need it
SNe_names = ('2011fe 2006D 2007A 2005W 1999dq 2009ig 2002fk 2012fr 2001el 2021pit 2005df 2005df_ANU 2015F 2018gv 2001bg 1995al 1997bq '
            +'2008fv 2008fv_comb 2021hpr 2019np 1994ae 2012ht 2015so ASASSN-15so 2011by 1998aq 2007sr Anchor 2012cg 1981B 1990N 1997bp 1999cp 2002cr '
            +'2007af 2013aa 2017cbv 2009Y 2017erp 2005cf 2013dy 2006bh 1998dh 2002dp 2003du').split()

# load the SNe in Hubble flow from Pantheon+
SNe_file = 'data/Pantheon+SH0ES.dat'
t = Table.read(SNe_file, format='ascii')

# indices of Riess baseline analysis
HF = np.where(t['USED_IN_SH0ES_HF'])
# indices of Riess analysis variant 34 (most of Pantheon+)
Panth = np.where((t['zHD'] > 0.0233) & (t['zHD'] < 0.8) & (t['IS_CALIBRATOR'] == False))
# indices of Cepheid host SNe (calibrators)
cal = np.where(t['IS_CALIBRATOR'])

# q0 and j0 from Riess 2016 pg. 11
q0 = -0.55
j0 = 1

# y vector entries
HF_ydata = t[HF]['m_b_corr'] - 5 * np.log10(299792.458*t[HF]['zHD'] * (1 + (1/2) * (1 - q0) * t[HF]['zHD'] - (1/6) * (1 - q0 - 3 * q0**2 + j0) * t[HF]['zHD']**2)) - 25
Panth_ydata = t[Panth]['m_b_corr'] - 5 * np.log10(299792.458*t[Panth]['zHD'] * (1 + (1/2) * (1 - q0) * t[Panth]['zHD'] - (1/6) * (1 - q0 - 3 * q0**2 + j0) * t[Panth]['zHD']**2)) - 25
cal_ydata = t[cal]['m_b_corr']
    
# covariance matrix (just get entries for the SNe we are using)
file = 'data/Pantheon+SH0ES_STAT+SYS.cov'
cov = np.loadtxt(file)[1:]
cov = cov.reshape(len(t), len(t))
# start getting the covariance entries. also will need covariance between calibrators and Pantheon SNe
HF_cov = cov[np.ix_(HF[0], HF[0])]
Panth_cov = cov[np.ix_(Panth[0], Panth[0])]
cal_cov = cov[np.ix_(cal[0], cal[0])]

In [126]:
# Riess data vector
Y_fits_path = 'data/ally_shoes_ceph_topantheonwt6.0_112221.fits'
Y = fits.open(Y_fits_path)[0].data
# Riess equation matrix 
L_fits_path = 'data/alll_shoes_ceph_topantheonwt6.0_112221.fits'
L = fits.open(L_fits_path)[0].data
# Riess covariance matrix
C_fits_path = 'data/allc_shoes_ceph_topantheonwt6.0_112221.fits'
C = fits.open(C_fits_path)[0].data

n1 = 3215 # index of first SNe entry
n2 = 3130 # index of first calibrator SNe
n3 = 3207 # index of last calibrator SNe (not inclusive -- last is 3206)

In [127]:
# need to (try to) match the Pantheon and Riess data
# variance and magnitude arrays for Riess and Pantheon catalogs
c_riess = np.array([C[i, i] for i in range(n2, n3)])
c_panth = np.array([cov[i, i] for i in cal])[0]
y_riess = np.array(Y[n2:n3])
y_panth = np.array(t[cal]['m_b_corr'])

# calculate the distance between the Riess magnitude and Pantheon magnitudes
magnitude_distances = np.abs(y_panth[:, np.newaxis] - y_riess[np.newaxis, :])

# calculate the distance between the Riess variance and Pantheon variance
covariance_distances = np.abs(c_panth[:, np.newaxis] - c_riess[np.newaxis, :])

# normalize the distances
normalized_magnitude_distances = magnitude_distances**2 / np.median(magnitude_distances)**2
normalized_covariance_distances = covariance_distances**2 / np.median(covariance_distances)**2

# combine the two normalized distances into a total distance matrix
total_distances = normalized_magnitude_distances + normalized_covariance_distances

# use Hungarian algorithm to find the best matching indices that minimize the total distance
row_indices, col_indices = linear_sum_assignment(total_distances.T)

# col_indices --> indices to match them

In [128]:
# Y VECTOR
# prepare data vectors of the correct size
HF_Y = np.zeros(n1 + len(HF_ydata))
Panth_Y = np.zeros(n1 + len(Panth_ydata))

# duplicate entreis for non-SNE entries in Y vector
HF_Y[:n1] = Y[:n1]
Panth_Y[:n1] = Y[:n1]

# replace calibrator entries in Y vector
HF_Y[n2:n3] = cal_ydata[col_indices]
Panth_Y[n2:n3] = cal_ydata[col_indices]

# add new data entries from Pantheon+ to Y vector
HF_Y[n1:] = HF_ydata
Panth_Y[n1:] = Panth_ydata

In [129]:
# C MATRIX
# prepare covariance matrices of the correct size
HF_C = np.zeros((n1 + len(HF_ydata), n1 + len(HF_ydata)))
Panth_C = np.zeros((n1 + len(Panth_ydata), n1 + len(Panth_ydata)))

# duplicate entries for non-SNe from Riess paper covariance
HF_C[:n1, :n1] = C[:n1, :n1]
Panth_C[:n1, :n1] = C[:n1, :n1]

# replace calibrator entries in covariance matrix
HF_C[n2:n3, n2:n3] = cal_cov[np.ix_(col_indices, col_indices)]
Panth_C[n2:n3, n2:n3] = cal_cov[np.ix_(col_indices, col_indices)]

# add new covariance entries from Pantheon+ non-calibrators
HF_C[n1:, n1:] = HF_cov
Panth_C[n1:, n1:] = Panth_cov

# get covariance between Pantheon+ and calibrators
HF_oddcov = cov[np.ix_(HF[0], cal[0][col_indices])]
Panth_oddcov = cov[np.ix_(Panth[0], cal[0][col_indices])]

# insert these covariance terms into the overall matrices
HF_C[n2:n3, n1:] = HF_oddcov.T
HF_C[n1:, n2:n3] = HF_oddcov
Panth_C[n2:n3, n1:] = Panth_oddcov.T
Panth_C[n1:, n2:n3] = Panth_oddcov

In [148]:
# L MATRIX
# HF has same equation matrix as Riess
HF_L = L

# Need to add rows to the Pantheon L matrix since we have more SNe
Panth_L = np.zeros((L.shape[0], n1+len(Panth_ydata)))
Panth_L[:,:n1] = L[:,:n1]
Panth_L[:,n1:] = L[:,-1][:,np.newaxis] * np.ones(len(Panth_ydata))[np.newaxis,:]

In [104]:
cal_cov_riess = C[3130:3207, 3130:3207]
HF_cov_riess = C[3215:, 3215:]

In [241]:
cal_cov[:2,:2]

array([[0.03177108, 0.00575443],
       [0.00575443, 0.03456656]])

In [239]:
C[3130:3132,3130:3132]

array([[0.01419299, 0.00406494],
       [0.00406494, 0.01691981]], dtype='>f4')

In [209]:
t[0]

CID,IDSURVEY,zHD,zHDERR,zCMB,zCMBERR,zHEL,zHELERR,m_b_corr,m_b_corr_err_DIAG,MU_SH0ES,MU_SH0ES_ERR_DIAG,CEPH_DIST,IS_CALIBRATOR,USED_IN_SH0ES_HF,c,cERR,x1,x1ERR,mB,mBERR,x0,x0ERR,COV_x1_c,COV_x1_x0,COV_c_x0,RA,DEC,HOST_RA,HOST_DEC,HOST_ANGSEP,VPEC,VPECERR,MWEBV,HOST_LOGMASS,HOST_LOGMASS_ERR,PKMJD,PKMJDERR,NDOF,FITCHI2,FITPROB,m_b_corr_err_RAW,m_b_corr_err_VPEC,biasCor_m_b,biasCorErr_m_b,biasCor_m_b_COVSCALE,biasCor_m_b_COVADD
str15,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64
2011fe,51,0.00122,0.00084,0.00122,2e-05,0.00082,2e-05,9.74571,1.51621,28.9987,1.51645,29.177,1,0,-0.1076,0.04008,-0.548188,0.13373,9.58436,0.0327221,2.63181,0.0793177,0.00011378,-0.00052525,-0.00272765,210.774,54.2737,-999,-999,-9.0,0.0,250,0.00758935,10.677,-9.0,55815.0,0.1071,36,26.8859,0.86447,0.0991,1.496,0.0381,0.005,1.0,0.003


In [208]:
cal_cov[0,0]

np.float64(0.03177108)

In [161]:
L[0:5,300:305].T

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]], dtype='>f4')